In [15]:
import time

print("Last updated : {}".format(time.strftime("%Y.%m.%d %H:%M:%S")))

Last updated : 2018.02.22 22:59:54


# TensorFlow 기본 원리

+ (1) 그래프 준비하기 (construction)
+ (2) 그래프 실행하기 (execution)

---

# MNIST 기본 세팅

**필요한 모듈 임포트**

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

**tf 버전 확인**

In [2]:
tf.__version__

'1.3.0'

**mnist 데이터 다운로드**

In [3]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
print(len(mnist.train.images[0])) # 784
print(len(mnist.train.labels[0])) # 10

784
10


---

# 4) CNN

## Part I. 그래프 그리기

+ 계산 그래프 그림

**00 random seed**

In [5]:
tf.set_random_seed(777)  # reproducibility

In [6]:
tf.reset_default_graph()

**01 placeholder**

In [7]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1]) # -1 means None
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

**02 predictions**

**conv layer 1**

In [8]:
W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01)) # filter_height, fliter_width, in_channel, out_channel(= # of filters)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME") # strides=[1, stride, stride, 1]
print(L1)
L1 = tf.nn.relu(L1)
print(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME") # ksize=[1, filter_h, filter_w, 1]
print(L1)

Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


**conv layer 2**

In [9]:
W2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01)) # filter_height, fliter_width, in_channel, out_channel(= # of filters)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME") # strides=[1, stride, stride, 1]
print(L2)
L2 = tf.nn.relu(L2)
print(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME") # ksize=[1, filter_h, filter_w, 1]
print(L2)
L2 = tf.reshape(L2, [-1, 7 * 7 * 64]) # for input of fully-connected layer
print(L2)

Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)


**fc layer**

In [10]:
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], initializer=tf.contrib.layers.xavier_initializer())
b= tf.Variable(tf.random_normal([10]))
predictions = tf.matmul(L2, W3) + b

---

**03 cost function and optimizer**

In [11]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

**04 accuracy**

In [12]:
is_correct = tf.equal(tf.argmax(predictions , axis=1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

## Part II. 그래프 실행

**05 training epoch/batch**

In [13]:
epochs = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))

print("Learning finished!")

Learning started. It takes sometime.
Epoch: 0001 cost = 0.167677274
Epoch: 0002 cost = 0.051725591
Epoch: 0003 cost = 0.039697459
Epoch: 0004 cost = 0.035156701
Epoch: 0005 cost = 0.028562424
Epoch: 0006 cost = 0.023723662
Epoch: 0007 cost = 0.024250871
Epoch: 0008 cost = 0.021302621
Epoch: 0009 cost = 0.019063985
Epoch: 0010 cost = 0.019105833
Epoch: 0011 cost = 0.018277362
Epoch: 0012 cost = 0.014793184
Epoch: 0013 cost = 0.018091603
Epoch: 0014 cost = 0.014925013
Epoch: 0015 cost = 0.013527846
Learning finished!


**06 test accuracy**

In [14]:
print("Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Accuracy: 0.9859


---